# Q1

In [1]:
import json

with open('/workspaces/llm-zoomcamp-own/01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

# Q2

In [2]:
from sentence_transformers import SentenceTransformer

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer("all-MiniLM-L12-v2")

# Q3

## Get the dimension of the model embedding

In [4]:
len(model.encode("This is a simple sentence"))

384

In [5]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

In [6]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '81fa104a888e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Pkyoh6YVT2WyATy1bolQsw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Q4

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Q5

In [9]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])


In [10]:
res["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'cLeuhZAB6cCxib9SQxws',
 '_score': 0.852048,
 '_source': {'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}}

# Q6

In [11]:
es_client.indices.get_mapping(index=index_name)

ObjectApiResponse({'course-questions': {'mappings': {'properties': {'course': {'type': 'keyword'}, 'question': {'type': 'text'}, 'question_vector': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine'}, 'section': {'type': 'keyword'}, 'text': {'type': 'text'}}}}})

In [13]:
knn_query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

response["hits"]["hits"]

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: field [text_vector] does not exist in the mapping')

In [15]:
# Define the new index mapping
new_index_mapping = {
  "mappings": {
    "properties": {
      "text_vector": {
        "type": "dense_vector",
        "dims": 128  # Adjust the dimension size according to your vector size
      },
      "section": {
        "type": "text"
      }
      # Include other fields as necessary
    }
  }
}

# Create the new index with the mapping
es_client.indices.create(index="new_index_name", body=new_index_mapping)

# You will then need to re-index your data from the old index to this new index

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'new_index_name'})

In [16]:
response = es_client.search(
    index="new_index_name",  # Update to the new index name
    body={
        "query": {
            "match": {"section": "General course-related questions"}
        },
        "knn": {
            "field": "text_vector",
            "query_vector": vector_search_term,
            "k": 5,
            "num_candidates": 10000
        },
        "size": 5
    }
)

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: to perform knn search on field [text_vector], its mapping must have [index] set to [true]')

In [17]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

# Assuming 'model.encode' is defined elsewhere and 'operations' is a list of documents to index
# from your_model import model
# operations = [...]

es_client.info()
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "question_vector": {"type": "dense_vector", "dims": 384}
        }
    }
}

index_name = "course-questions"

# Delete the index if it exists and create a new one with the specified settings
es_client.indices.delete(index=index_name, ignore=[400, 404])
es_client.indices.create(index=index_name, body=index_settings)

# Index documents
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Encode the search term to a vector
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

# Construct the KNN query using a script score query
knn_query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'question_vector') + 1.0",
                "params": {"query_vector": vector_search_term}
            }
        }
    },
    "_source": ["text", "section", "question", "course"],
    "size": 5
}

# Execute the search query
response = es_client.search(index=index_name, body=knn_query)

# Print the hits


/tmp/ipykernel_27142/3035893565.py:28: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=index_name, ignore=[400, 404])


[{'_index': 'course-questions', '_id': 'JLe1hZAB6cCxib9S-yCP', '_score': 1.704096, '_source': {'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp', 'section': 'General course-related questions', 'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}}, {'_index': 'course-questions', '_id': 'Rbe1hZAB6cCxib9S_iCJ', '_score': 1.6763345, '_source': {'question': 'How can we contribute to the course?', 'course': 'data-engineering-zoomcamp', 'section': 'General course-related questions', 'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'}}, {'_index': 'course-questions', '_id': '57e2hZAB6cCxib9SIyGt', '_score': 1.6568459, '_source': {'question': 'I just joined. What should I do next? How can I access course materials?', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions', 'text': 'Welc

In [39]:
response["hits"]["hits"][0]['_source']['section']

'General course-related questions'

In [40]:
section_list = set()
for i in range(len(response["hits"]["hits"])):
    section_list.add(response["hits"]["hits"][0]['_source']['section'])
section_list

{'General course-related questions'}